In [ ]:
import pandas as pd
import olca_ipc as ipc
import olca_schema as schema

client = ipc.Client(8080)
client

In [ ]:
# Archivo donde guardaremos los procesos
FILE_PATH = "process_descriptors.txt"

# Leer los IDs ya guardados (si el archivo existe)
try:
    with open(FILE_PATH, "r", encoding="utf-8") as f:
        saved_ids = {line.split("\t")[0] for line in f if line.strip()}
except FileNotFoundError:
    saved_ids = set()

print(f"Ya tienes {len(saved_ids)} procesos guardados.")

In [ ]:
descriptors = client.get_descriptors(schema.Process)

# Contar cuántos hay
total_procesos = len(descriptors)
print(f"Total de procesos en la base: {total_procesos}")

In [ ]:
 3. Reanudar desde los que faltan
with open(FILE_PATH, "a", encoding="utf-8") as f:
    for i, d in enumerate(descriptors, start=1):
        if d.id in saved_ids:
            continue  
        try:
            proceso = client.get(schema.Process, d.id)
            f.write(f"{proceso.id}\t{proceso.name}\t{proceso.category}\n")
            saved_ids.add(proceso.id)

            if len(saved_ids) % 100 == 0:
                print(f"Van {len(saved_ids)} procesos guardados...")
        except Exception as e:
            print(f" Error en {d.id}: {e}")
            break

In [ ]:
FILE_PATH = "process_descriptors.txt"  
df = pd.read_csv(FILE_PATH, sep="\t", names=["id", "name", "category"], header=None)
prefijo = "H:Transportation and storage" #Cambiar el nombre ###No quitar el ":"
filtered = df[df["category"].str.startswith(prefijo, na=False)]


_process_ids = filtered["id"].tolist()

print(f"Se encontraron {len(_process_ids)} procesos bajo la categoría {prefijo}")
print(_process_ids[:5])  # muestra solo los primeros 10

In [ ]:
import pandas as pd
import json
import os

impact_data = []

# Archivos de salida
FILE_PATH = "H Transportation and storage.jsonl"   # resultados ##Cambiar el nombre ##Quitar ":"
CHECKPOINT_FILE = "checkpoint.txt"   # último proceso exitoso ##Borarr el documento al empezar un nuevo documento
ERROR_FILE = "errores.txt"           # log de errores ##Borrar el documento al empezar una nueva carpeta

impact_method_ref = client.get_descriptor(
    schema.ImpactMethod,
    '61966689-76aa-4b3b-94f1-81989199433f'
)

#  Leer checkpoint si existe
start_index = 0
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, "r") as f:
        try:
            start_index = int(f.read().strip())
        except:
            start_index = 0

print(f" Retomando desde el índice {start_index}")

# Iterar sobre los procesos
for idx, process_id in enumerate(_process_ids[start_index:], start=start_index):
    try:
        process_ref = client.get_descriptor(schema.Process, process_id)

        # Configurar el sistema de producto
        config = schema.LinkingConfig(
            prefer_unit_processes=True,
            provider_linking=schema.ProviderLinking.ONLY_DEFAULTS
        )
        product_system_ref = client.create_product_system(process_ref, config)

        # Configurar el cálculo
        setup = schema.CalculationSetup(
            target=product_system_ref,
            impact_method=impact_method_ref
        )
        result = client.calculate(setup)
        state = result.wait_until_ready()

        # Guardar resultados en JSONL (línea por línea)
        with open(FILE_PATH, "a", encoding="utf-8") as f:
            for impact in result.get_total_impacts():
                record = {
                    "Name": process_ref.name,
                    "Category": impact.impact_category.name,
                    "Unit": impact.impact_category.ref_unit,
                    "Amount": impact.amount,
                    "ubi": process_ref.location,
                    "id": process_id
                }
                f.write(json.dumps(record, ensure_ascii=False) + "\n")

        result.dispose()
        client.delete(product_system_ref)

        # Guardar progreso en el checkpoint
        with open(CHECKPOINT_FILE, "w") as f:
            f.write(str(idx + 1))

    except Exception as e:
        # Guardar errores para revisar después
        with open(ERROR_FILE, "a", encoding="utf-8") as ef:
            ef.write(f"Error en proceso {process_id} (índice {idx}): {str(e)}\n")
        print(f" Error en proceso {process_id}, registrado en {ERROR_FILE}")
        continue

In [ ]:
# Cargar JSONL a DataFrame
df = pd.read_json("H Transportation and storage.jsonl", lines=True, encoding="utf-8")

# Ver las primeras filas
#print(df.head())

df

In [ ]:
# Pivotear
final_df = df.pivot_table(
    index=["Name", "id", "ubi"],   
    columns="Category",
    values="Amount",
    aggfunc="sum",
    fill_value=0
).reset_index()


print(final_df)
# Exportar a Excel
final_df.to_excel("H Transportation and storage.xlsx", index=False)